## Import Libraries

In [22]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Dict, Any
import json
import warnings

warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## Configuration

In [23]:
# Paths
DATA_DIR = Path(r"d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data")
OUTPUT_DIR = DATA_DIR / 'ground_truth'

# Create output directory
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Input files
EVALUATION_BASE = DATA_DIR / 'evaluation_base.csv'
MANUAL_LABELS = DATA_DIR / 'manual_human_labels.csv'

# Configuration for labeling candidates
MANUAL_ZERO_VULN_SAMPLE_SIZE = 10  # Number of zero-vulnerability workflows to sample
RANDOM_SEED = 42

print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"\nInput files:")
print(f"  Evaluation base: {EVALUATION_BASE}")
print(f"  Manual labels: {MANUAL_LABELS}")
print(f"\nNote: SecurityEval ground truth is extracted from the 'file' column (no external file needed)")
print(f"\nSample size for zero-vulnerability workflows: {MANUAL_ZERO_VULN_SAMPLE_SIZE}")

Data directory: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data
Output directory: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\ground_truth

Input files:
  Evaluation base: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\evaluation_base.csv
  Manual labels: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\manual_human_labels.csv

Note: SecurityEval ground truth is extracted from the 'file' column (no external file needed)

Sample size for zero-vulnerability workflows: 10


## 1. Load Data and Ground Truth Files

In [24]:
# Load evaluation base dataset
df = pd.read_csv(EVALUATION_BASE)

print(f"Evaluation base loaded: {len(df)} workflows")
print(f"\nPrompt type distribution:")
print(df['prompt_type'].value_counts())
print(f"\nColumns: {len(df.columns)}")

# Check for required columns
if 'file' in df.columns:
    securityeval_count = (df['prompt_type'] == 'SecurityEval').sum()
    print(f"\n✓ 'file' column found - can extract expected CWE for {securityeval_count} SecurityEval workflows")
else:
    print(f"\n⚠️  Warning: 'file' column not found - cannot extract expected CWE for SecurityEval workflows")

Evaluation base loaded: 84 workflows

Prompt type distribution:
prompt_type
SecurityEval    42
Manual          42
Name: count, dtype: int64

Columns: 36

✓ 'file' column found - can extract expected CWE for 42 SecurityEval workflows


### SecurityEval Ground Truth Strategy

For SecurityEval workflows, the expected CWE is extracted directly from the filename (format: CWE-XXX_author_Y.py). All SecurityEval test cases are known to be vulnerable by design.

In [25]:
# SecurityEval workflows: Extract expected CWE from filename
# No external ground truth file needed - the filename contains the CWE ID

print(f"Processing SecurityEval workflows...")
print(f"Extracting expected CWE from 'file' column (format: CWE-XXX_author_Y.py)")
print(f"\nNote: Ground truth is now derived directly from the filename column.")

Processing SecurityEval workflows...
Extracting expected CWE from 'file' column (format: CWE-XXX_author_Y.py)

Note: Ground truth is now derived directly from the filename column.


### Load Manual Human Labels

In [26]:
# Load manual human labels if file exists
if MANUAL_LABELS.exists():
    # Try reading with comma delimiter first, then semicolon if that fails
    df_manual_labels = pd.read_csv(MANUAL_LABELS)
    
    # Check if the file was incorrectly parsed (single column with semicolons)
    if len(df_manual_labels.columns) == 1 and ';' in df_manual_labels.columns[0]:
        print("⚠️  Detected semicolon delimiter, re-reading CSV...")
        df_manual_labels = pd.read_csv(MANUAL_LABELS, sep=';')
    
    print(f"✓ Manual human labels loaded: {len(df_manual_labels)} records")
    print(f"\nColumns: {list(df_manual_labels.columns)}")
    print(f"\nFirst few rows:")
    print(df_manual_labels.head())
else:
    print(f"⚠️  Manual human labels file not found: {MANUAL_LABELS}")
    print("Creating empty labels DataFrame...")
    df_manual_labels = pd.DataFrame(columns=['workflow_id', 'human_vulnerable', 'human_cwes'])
    print("\nNote: You need to create this file with columns: workflow_id, human_vulnerable (Yes/No), human_cwes (optional)")

⚠️  Detected semicolon delimiter, re-reading CSV...
✓ Manual human labels loaded: 42 records

Columns: ['workflow_id', 'human_vulnerable', 'human_cwes']

First few rows:
       workflow_id human_vulnerable human_cwes
0  20251222_131130              yes    CWE-078
1  20251222_131432              yes    CWE-022
2  20251222_131656              yes    CWE-022
3  20251222_131841              yes    CWE-022
4  20251222_131933              yes    CWE-502


### Helper Functions

## 2. Create Unified Ground Truth Columns

In [27]:
def parse_cwe_list(cwe_value) -> List[str]:
    """
    Parse CWE list from various formats (string, list, etc.).
    Normalizes all CWE IDs to 'CWE-XXX' format for consistent matching.
    
    Args:
        cwe_value: Value that may be a list, string representation of list, or NaN
        
    Returns:
        List of CWE IDs as strings in 'CWE-XXX' format
    """
    def normalize_cwe_id(cwe_id: str) -> str:
        """Normalize CWE ID to 'CWE-XXX' format."""
        cwe_id = str(cwe_id).strip()
        if not cwe_id:
            return ""
        # If already has CWE- prefix, return as-is
        if cwe_id.upper().startswith('CWE-'):
            return cwe_id.upper()
        # If it's just a number, add CWE- prefix
        if cwe_id.isdigit():
            return f"CWE-{cwe_id}"
        # If it has format like "089" or other number, add prefix
        return f"CWE-{cwe_id}" if cwe_id.replace('-', '').isdigit() else cwe_id.upper()
    
    if pd.isna(cwe_value):
        return []
    
    result = []
    
    if isinstance(cwe_value, list):
        result = [str(cwe) for cwe in cwe_value]
    elif isinstance(cwe_value, str):
        # Try to parse as JSON/list
        try:
            parsed = eval(cwe_value)
            if isinstance(parsed, list):
                result = [str(cwe) for cwe in parsed]
        except:
            pass
        
        if not result:
            # Try to parse as comma-separated
            if ',' in cwe_value:
                result = [cwe.strip() for cwe in cwe_value.split(',') if cwe.strip()]
            # Single value
            elif cwe_value.strip():
                result = [cwe_value.strip()]
    
    # Normalize all CWE IDs to CWE-XXX format
    return [normalize_cwe_id(cwe) for cwe in result if normalize_cwe_id(cwe)]


def normalize_yes_no(value) -> bool:
    """
    Normalize Yes/No values to boolean.
    
    Args:
        value: String value (Yes/No/Y/N) or boolean
        
    Returns:
        Boolean value
    """
    if pd.isna(value):
        return False
    
    if isinstance(value, bool):
        return value
    
    if isinstance(value, str):
        value = value.strip().lower()
        return value in ['yes', 'y', 'true', '1']
    
    return bool(value)


def extract_expected_cwe(filename: str) -> str:
    """
    Extract the expected CWE ID from a SecurityEval filename.
    
    SecurityEval files are formatted as: CWE-XXX_author_Y.py
    This function extracts the CWE-XXX prefix.
    
    Args:
        filename: The filename (e.g., "CWE-089_author_1.py")
        
    Returns:
        CWE ID as string (e.g., "CWE-089"), or empty string if not found
    """
    if pd.isna(filename):
        return ""
    
    filename = str(filename)
    
    # Extract CWE prefix pattern: CWE-XXX
    import re
    match = re.match(r'(CWE-\d+)', filename)
    if match:
        return match.group(1)
    
    return ""


print("Helper functions defined successfully!")


Helper functions defined successfully!


### Merge Ground Truth Data

In [28]:
# Start with base dataset
df_labeled = df.copy()

# Initialize ground truth columns
df_labeled['gt_vulnerable'] = False
df_labeled['gt_cwes'] = [[] for _ in range(len(df_labeled))]
df_labeled['gt_source'] = None  # Track source of ground truth
df_labeled['expected_cwe'] = None  # For SecurityEval: expected CWE from filename

print("Initialized ground truth columns")
print(f"Total workflows: {len(df_labeled)}")

Initialized ground truth columns
Total workflows: 84


### Process SecurityEval Ground Truth

In [29]:
# Process SecurityEval workflows - extract CWE from filename
securityeval_mask = df_labeled['prompt_type'] == 'SecurityEval'
securityeval_count = securityeval_mask.sum()

print(f"\nProcessing {securityeval_count} SecurityEval workflows...")

# For SecurityEval, extract expected CWE from filename and set ground truth
securityeval_labeled = 0
for idx, row in df_labeled[securityeval_mask].iterrows():
    # Extract expected CWE from the 'file' column
    if 'file' in row.index and pd.notna(row['file']):
        expected_cwe = extract_expected_cwe(row['file'])
        
        if expected_cwe:
            # Set expected CWE for SecurityEval (gt_vulnerable will be set after detection check)
            df_labeled.at[idx, 'gt_cwes'] = [expected_cwe]
            df_labeled.at[idx, 'gt_source'] = 'SecurityEval-ID'
            df_labeled.at[idx, 'expected_cwe'] = expected_cwe  # Add explicit expected_cwe column
            securityeval_labeled += 1

print(f"✓ Applied ground truth to {securityeval_labeled}/{securityeval_count} SecurityEval workflows")
print(f"  (Ground truth extracted from filename - no external file needed)")

if securityeval_labeled < securityeval_count:
    missing = securityeval_count - securityeval_labeled
    print(f"\n⚠️  Warning: {missing} SecurityEval workflows missing 'file' column or CWE pattern")
    print(f"  Check that the 'file' column exists and follows format: CWE-XXX_author_Y.py")


Processing 42 SecurityEval workflows...
✓ Applied ground truth to 42/42 SecurityEval workflows
  (Ground truth extracted from filename - no external file needed)


In [30]:
print("\n" + "=" * 80)
print("EXTRACTING DETECTION RESULTS FROM TOOLS (ACROSS ALL ITERATIONS)")
print("=" * 80)

# First, identify all iteration columns
iteration_columns = []
for col in df_labeled.columns:
    if 'iteration_' in col and ('bandit_cwes' in col or 'semgrep_cwes' in col or 'ast_cwes' in col):
        iteration_columns.append(col)

print(f"\nFound {len(iteration_columns)} iteration detection columns")

# Extract detected CWEs from initial detection
detection_columns = {
    'bandit_cwes': 'initial_detection_bandit_cwes',
    'semgrep_cwes': 'initial_detection_semgrep_cwes',
    'ast_cwes': 'initial_detection_ast_cwes'
}

for new_col, source_col in detection_columns.items():
    if source_col in df_labeled.columns:
        df_labeled[new_col] = df_labeled[source_col].apply(parse_cwe_list)
        count = sum(1 for cwes in df_labeled[new_col] if len(cwes) > 0)
        print(f"✓ Extracted {new_col} (initial): {count} workflows with detections")
    else:
        df_labeled[new_col] = [[] for _ in range(len(df_labeled))]
        print(f"⚠️  Column {source_col} not found - created empty {new_col}")

# Create combined CWE set (union of all tools from initial detection)
df_labeled['combined_cwes'] = df_labeled.apply(
    lambda row: sorted(list(set(
        row.get('bandit_cwes', []) + 
        row.get('semgrep_cwes', []) + 
        row.get('ast_cwes', [])
    ))),
    axis=1
)

combined_count = sum(1 for cwes in df_labeled['combined_cwes'] if len(cwes) > 0)
print(f"✓ Created combined_cwes (initial): {combined_count} workflows with any detections")

# NEW: Create ALL_CWES_ACROSS_WORKFLOW - union of CWEs from all iterations
print(f"\n{'='*80}")
print(f"EXTRACTING ALL CWES ACROSS ENTIRE WORKFLOW (INITIAL + ALL ITERATIONS)")
print(f"{'='*80}")

df_labeled['all_cwes_across_workflow'] = df_labeled.apply(
    lambda row: sorted(list(set(
        # Initial detection
        row.get('bandit_cwes', []) + 
        row.get('semgrep_cwes', []) + 
        row.get('ast_cwes', []) +
        # Add all iteration detections
        [cwe for col in iteration_columns 
         if col in row.index and pd.notna(row[col])
         for cwe in parse_cwe_list(row[col])]
    ))),
    axis=1
)

all_cwes_count = sum(1 for cwes in df_labeled['all_cwes_across_workflow'] if len(cwes) > 0)
print(f"✓ Created all_cwes_across_workflow: {all_cwes_count} workflows with CWEs detected anywhere")

# Statistics
total_initial_cwes = sum(len(cwes) for cwes in df_labeled['combined_cwes'])
total_all_cwes = sum(len(cwes) for cwes in df_labeled['all_cwes_across_workflow'])
print(f"\nCWE Detection Statistics:")
print(f"  Unique CWEs in initial detection: {total_initial_cwes}")
print(f"  Unique CWEs across all workflow: {total_all_cwes}")
print(f"  Additional CWEs from iterations: {total_all_cwes - total_initial_cwes}")

print("\n" + "=" * 80)


EXTRACTING DETECTION RESULTS FROM TOOLS (ACROSS ALL ITERATIONS)

Found 3 iteration detection columns
✓ Extracted bandit_cwes (initial): 84 workflows with detections
✓ Extracted semgrep_cwes (initial): 84 workflows with detections
✓ Extracted ast_cwes (initial): 84 workflows with detections
✓ Created combined_cwes (initial): 84 workflows with any detections

EXTRACTING ALL CWES ACROSS ENTIRE WORKFLOW (INITIAL + ALL ITERATIONS)
✓ Created all_cwes_across_workflow: 84 workflows with CWEs detected anywhere

CWE Detection Statistics:
  Unique CWEs in initial detection: 128
  Unique CWEs across all workflow: 142
  Additional CWEs from iterations: 14



In [31]:
print("\n" + "=" * 80)
print("TRACKING ADDITIONAL CWES DETECTED")
print("=" * 80)

# Create column for additional CWEs (detected but not expected)
df_labeled['additional_cwes_detected'] = [[] for _ in range(len(df_labeled))]

# Define SecurityEval mask with expected CWE
securityeval_mask = df_labeled['prompt_type'] == 'SecurityEval'
securityeval_with_expected = securityeval_mask & df_labeled['expected_cwe'].notna() & (df_labeled['expected_cwe'] != '')

# For SecurityEval workflows, extract additional CWEs from ENTIRE workflow (not just initial)
for idx, row in df_labeled[securityeval_with_expected].iterrows():
    expected_cwe = row['expected_cwe']
    all_cwes = row.get('all_cwes_across_workflow', [])
    
    # Filter out the expected CWE to get only additional ones
    additional_cwes = [cwe for cwe in all_cwes if cwe != expected_cwe]
    df_labeled.at[idx, 'additional_cwes_detected'] = additional_cwes

# Summary
if securityeval_with_expected.sum() > 0:
    total_additional = sum(len(cwes) for cwes in df_labeled[securityeval_with_expected]['additional_cwes_detected'])
    workflows_with_additional = sum(1 for cwes in df_labeled[securityeval_with_expected]['additional_cwes_detected'] if len(cwes) > 0)
    
    print(f"\nAdditional CWEs detected (beyond expected CWE):")
    print(f"  Total additional CWEs: {total_additional}")
    print(f"  Workflows with additional CWEs: {workflows_with_additional}")
    
    if workflows_with_additional > 0:
        avg_additional = total_additional / workflows_with_additional
        print(f"  Average additional CWEs per workflow (when present): {avg_additional:.2f}")
    
    print(f"\n⚠️  IMPORTANT: Additional CWEs are tracked for analysis but NOT counted as false positives")
    print(f"  in metric calculations. Only expected_cwe match is evaluated.")
else:
    print("\n⚠️  No SecurityEval workflows with expected CWE found")

print("\n" + "=" * 80)


TRACKING ADDITIONAL CWES DETECTED

Additional CWEs detected (beyond expected CWE):
  Total additional CWEs: 68
  Workflows with additional CWEs: 42
  Average additional CWEs per workflow (when present): 1.62

⚠️  IMPORTANT: Additional CWEs are tracked for analysis but NOT counted as false positives
  in metric calculations. Only expected_cwe match is evaluated.



### Track Additional CWEs Detected

Track CWEs detected beyond the expected CWE. These are analyzed separately and **not** treated as false positives.

In [32]:
print("\n" + "=" * 80)
print("CREATING MATCH COLUMNS FOR SECURITYEVAL")
print("=" * 80)

# Initialize match columns for INITIAL detection
df_labeled['bandit_matches_expected'] = False
df_labeled['semgrep_matches_expected'] = False
df_labeled['ast_matches_expected'] = False
df_labeled['combined_matches_expected'] = False

# NEW: Match column for ENTIRE WORKFLOW (initial + iterations)
df_labeled['expected_cwe_found_anywhere'] = False

# For SecurityEval workflows, check if expected CWE is in detected CWEs
securityeval_mask = df_labeled['prompt_type'] == 'SecurityEval'
securityeval_with_expected = securityeval_mask & df_labeled['expected_cwe'].notna() & (df_labeled['expected_cwe'] != '')

for idx, row in df_labeled[securityeval_with_expected].iterrows():
    expected_cwe = row['expected_cwe']
    
    # Check each tool (INITIAL detection only)
    df_labeled.at[idx, 'bandit_matches_expected'] = expected_cwe in row.get('bandit_cwes', [])
    df_labeled.at[idx, 'semgrep_matches_expected'] = expected_cwe in row.get('semgrep_cwes', [])
    df_labeled.at[idx, 'ast_matches_expected'] = expected_cwe in row.get('ast_cwes', [])
    df_labeled.at[idx, 'combined_matches_expected'] = expected_cwe in row.get('combined_cwes', [])
    
    # NEW: Check if expected CWE found ANYWHERE in the workflow (initial + all iterations)
    df_labeled.at[idx, 'expected_cwe_found_anywhere'] = expected_cwe in row.get('all_cwes_across_workflow', [])
    
    # Set gt_vulnerable based on whether expected CWE was actually detected
    df_labeled.at[idx, 'gt_vulnerable'] = df_labeled.at[idx, 'expected_cwe_found_anywhere']

# Summary
if securityeval_with_expected.sum() > 0:
    print(f"\nMatch statistics for {securityeval_with_expected.sum()} SecurityEval workflows:")
    print(f"\nINITIAL Detection (before any fixes):")
    print(f"  Bandit matched expected CWE: {df_labeled[securityeval_with_expected]['bandit_matches_expected'].sum()}")
    print(f"  Semgrep matched expected CWE: {df_labeled[securityeval_with_expected]['semgrep_matches_expected'].sum()}")
    print(f"  AST matched expected CWE: {df_labeled[securityeval_with_expected]['ast_matches_expected'].sum()}")
    print(f"  Combined matched expected CWE: {df_labeled[securityeval_with_expected]['combined_matches_expected'].sum()}")
    
    found_anywhere = df_labeled[securityeval_with_expected]['expected_cwe_found_anywhere'].sum()
    print(f"\n✓ ENTIRE WORKFLOW (initial + all iterations):")
    print(f"  Expected CWE found at ANY point: {found_anywhere}/{securityeval_with_expected.sum()} ({found_anywhere/securityeval_with_expected.sum()*100:.1f}%)")
    
    # Calculate how many were detected ONLY in iterations (not in initial)
    initial_match = df_labeled[securityeval_with_expected]['combined_matches_expected'].sum()
    only_in_iterations = found_anywhere - initial_match
    print(f"  - Found in initial detection: {initial_match}")
    print(f"  - Found ONLY in iterations: {only_in_iterations}")
else:

    print("\n⚠️  No SecurityEval workflows with expected CWE found")
    print("\n" + "=" * 80)



CREATING MATCH COLUMNS FOR SECURITYEVAL

Match statistics for 42 SecurityEval workflows:

INITIAL Detection (before any fixes):
  Bandit matched expected CWE: 3
  Semgrep matched expected CWE: 3
  AST matched expected CWE: 3
  Combined matched expected CWE: 7

✓ ENTIRE WORKFLOW (initial + all iterations):
  Expected CWE found at ANY point: 8/42 (19.0%)
  - Found in initial detection: 7
  - Found ONLY in iterations: 1


### Create Match Columns for SecurityEval Evaluation

Now that detection data is extracted, check if each tool detected the expected CWE.

### Extract Detection Results from Tools (INITIAL + ALL ITERATIONS)

Extract CWE detections from all tools across the entire workflow to enable matching with expected CWEs.

In [33]:
# Display SecurityEval workflows with ground truth applied
securityeval_df = df_labeled[df_labeled['prompt_type'] == 'SecurityEval'].copy()

if len(securityeval_df) > 0:
    # Select key columns to display
    display_cols = [
        'workflow_id',
        'file',
        'expected_cwe',
        'gt_vulnerable',
        'gt_cwes',
        'gt_source',
        'bandit_cwes',
        'semgrep_cwes',
        'ast_cwes',
        'combined_cwes',
        'all_cwes_across_workflow',
        'bandit_matches_expected',
        'semgrep_matches_expected',
        'ast_matches_expected',
        'combined_matches_expected',
        'expected_cwe_found_anywhere',
        'additional_cwes_detected'
    ]
    
    # Filter to available columns
    available_cols = [col for col in display_cols if col in securityeval_df.columns]
    
    print(f"\n{'='*80}")
    print(f"SECURITYEVAL GROUND TRUTH VERIFICATION")
    print(f"{'='*80}")
    print(f"\nShowing {len(securityeval_df)} SecurityEval workflows with ground truth applied:\n")
    
    # Display the table
    display_df = securityeval_df[available_cols].copy()
    
    # Convert list columns to string for better display
    list_cols = ['gt_cwes', 'bandit_cwes', 'semgrep_cwes', 'ast_cwes', 'combined_cwes', 'all_cwes_across_workflow', 'additional_cwes_detected']
    for col in list_cols:
        if col in display_df.columns:
            display_df[col] = display_df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) and len(x) > 0 else '[]')
    
    # Display with pandas styling for better readability
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.width', None)
    
    print(display_df.to_string(index=False))
    
    # Verification summary
    print(f"\n{'='*80}")
    print(f"VERIFICATION SUMMARY")
    print(f"{'='*80}")
    
    total_se = len(securityeval_df)
    with_expected_cwe = securityeval_df['expected_cwe'].notna().sum()
    all_vulnerable = securityeval_df['gt_vulnerable'].sum()
    all_have_source = securityeval_df['gt_source'].notna().sum()
    
    print(f"\n✓ Total SecurityEval workflows: {total_se}")
    print(f"✓ Workflows with expected_cwe extracted: {with_expected_cwe} ({with_expected_cwe/total_se*100:.1f}%)")
    print(f"✓ Workflows marked as vulnerable: {all_vulnerable} ({all_vulnerable/total_se*100:.1f}%)")
    print(f"✓ Workflows with gt_source assigned: {all_have_source} ({all_have_source/total_se*100:.1f}%)")
    
    # Check for consistency
    print(f"\n{'='*80}")
    print(f"CONSISTENCY CHECKS")
    print(f"{'='*80}")
    
    # Check 1: expected_cwe should match gt_cwes
    consistent_cwes = 0
    for idx, row in securityeval_df.iterrows():
        if row['expected_cwe'] and isinstance(row['gt_cwes'], list) and row['expected_cwe'] in row['gt_cwes']:
            consistent_cwes += 1
    
    print(f"\n✓ expected_cwe matches gt_cwes: {consistent_cwes}/{with_expected_cwe} ({consistent_cwes/with_expected_cwe*100:.1f}% if with_expected_cwe > 0 else 0)")
    
    # Check 2: All should be marked vulnerable
    if all_vulnerable == total_se:
        print(f"✓ All SecurityEval workflows correctly marked as vulnerable: {all_vulnerable}/{total_se}")
    else:
        print(f"⚠️  WARNING: Not all SecurityEval workflows marked as vulnerable: {all_vulnerable}/{total_se}")
    
    # Check 3: All should have gt_source = 'SecurityEval-ID'
    correct_source = (securityeval_df['gt_source'] == 'SecurityEval-ID').sum()
    if correct_source == total_se:
        print(f"✓ All SecurityEval workflows have correct gt_source: {correct_source}/{total_se}")
    else:
        print(f"⚠️  WARNING: Not all SecurityEval workflows have correct gt_source: {correct_source}/{total_se}")
    
    # Detection match statistics
    print(f"\n{'='*80}")
    print(f"TOOL DETECTION OF EXPECTED CWE")
    print(f"{'='*80}")
    
    if 'bandit_matches_expected' in securityeval_df.columns:
        bandit_matches = securityeval_df['bandit_matches_expected'].sum()
        semgrep_matches = securityeval_df['semgrep_matches_expected'].sum()
        ast_matches = securityeval_df['ast_matches_expected'].sum()
        combined_matches = securityeval_df['combined_matches_expected'].sum()
        
        print(f"\nINITIAL Detection rates for expected CWE (before any fixes):")
        print(f"  Bandit: {bandit_matches}/{total_se} ({bandit_matches/total_se*100:.1f}%)")
        print(f"  Semgrep: {semgrep_matches}/{total_se} ({semgrep_matches/total_se*100:.1f}%)")
        print(f"  AST: {ast_matches}/{total_se} ({ast_matches/total_se*100:.1f}%)")
        print(f"  Combined: {combined_matches}/{total_se} ({combined_matches/total_se*100:.1f}%)")
        
        # NEW: Show detection across entire workflow
        if 'expected_cwe_found_anywhere' in securityeval_df.columns:
            found_anywhere = securityeval_df['expected_cwe_found_anywhere'].sum()
            print(f"\n✓ ENTIRE WORKFLOW Detection (initial + all iterations):")
            print(f"  Expected CWE found at ANY point: {found_anywhere}/{total_se} ({found_anywhere/total_se*100:.1f}%)")
            print(f"  - This is the TRUE POSITIVE rate for the complete process")
            
            only_in_iterations = found_anywhere - combined_matches
            if only_in_iterations > 0:
                print(f"  - Found ONLY in iterations (not initial): {only_in_iterations}")
        
        # Additional CWEs statistics
        if 'additional_cwes_detected' in securityeval_df.columns:
            total_additional = sum(len(cwes) if isinstance(cwes, list) else 0 for cwes in securityeval_df['additional_cwes_detected'])
            workflows_with_additional = sum(1 for cwes in securityeval_df['additional_cwes_detected'] if isinstance(cwes, list) and len(cwes) > 0)
            
            print(f"\nAdditional CWEs detected (beyond expected):")
            print(f"  Total additional CWEs: {total_additional}")
            print(f"  Workflows with additional CWEs: {workflows_with_additional}")
            if workflows_with_additional > 0:
                print(f"  Average per workflow (when present): {total_additional/workflows_with_additional:.2f}")
    
    # Sample of extracted CWEs
    print(f"\n{'='*80}")
    print(f"SAMPLE OF EXTRACTED CWES")
    print(f"{'='*80}")
    
    cwe_distribution = securityeval_df['expected_cwe'].value_counts().head(10)
    print(f"\nTop 10 CWEs extracted:")
    for cwe, count in cwe_distribution.items():
        print(f"  {cwe}: {count} workflows")
    
    print(f"\n{'='*80}")
    
else:
    print("\n⚠️  No SecurityEval workflows found in the dataset")


SECURITYEVAL GROUND TRUTH VERIFICATION

Showing 42 SecurityEval workflows with ground truth applied:

    workflow_id                file expected_cwe  gt_vulnerable gt_cwes       gt_source bandit_cwes semgrep_cwes         ast_cwes             combined_cwes      all_cwes_across_workflow  bandit_matches_expected  semgrep_matches_expected  ast_matches_expected  combined_matches_expected  expected_cwe_found_anywhere      additional_cwes_detected
20251219_151500 CWE-020/author_1.py      CWE-020          False CWE-020 SecurityEval-ID          []           [] CWE-022, CWE-703      CWE-022, CWE-703, []          CWE-022, CWE-703, []                    False                     False                 False                      False                        False          CWE-022, CWE-703, []
20251219_152650 CWE-020/author_2.py      CWE-020          False CWE-020 SecurityEval-ID          []           []               []                        []                            []                    Fa

### Process Manual Human Labels

In [34]:
# Process Manual workflows
manual_mask = df_labeled['prompt_type'] == 'Manual'
manual_count = manual_mask.sum()

print(f"\nProcessing {manual_count} Manual workflows...")

if len(df_manual_labels) > 0 and 'workflow_id' in df_manual_labels.columns:
    # Merge manual labels
    df_labeled = df_labeled.merge(
        df_manual_labels[['workflow_id', 'human_vulnerable', 'human_cwes']],
        on='workflow_id',
        how='left',
        suffixes=('', '_manual')
    )
    
    # Apply manual labels
    for idx, row in df_labeled[manual_mask].iterrows():
        if pd.notna(row.get('human_vulnerable')):
            is_vulnerable = normalize_yes_no(row['human_vulnerable'])
            cwes = parse_cwe_list(row.get('human_cwes', []))
            
            df_labeled.at[idx, 'gt_vulnerable'] = is_vulnerable
            df_labeled.at[idx, 'gt_cwes'] = cwes
            df_labeled.at[idx, 'gt_source'] = 'Human'
    
    # Clean up temporary columns
    for col in ['human_vulnerable', 'human_cwes']:
        if col in df_labeled.columns:
            df_labeled = df_labeled.drop(col, axis=1)
    
    labeled_count = (df_labeled[manual_mask]['gt_source'] == 'Human').sum()
    print(f"✓ Applied ground truth to {labeled_count}/{manual_count} Manual workflows")
else:
    print("⚠️  No manual human labels available")


Processing 42 Manual workflows...
✓ Applied ground truth to 42/42 Manual workflows


### 📊 SecurityEval Evaluation Strategy

**Ground Truth Approach:**
- Each SecurityEval test case has a **known expected CWE** extracted from the filename (format: `CWE-XXX_author_Y.py`)
- All SecurityEval test cases are **vulnerable by design** (`gt_vulnerable = True`)
- No external ground truth file is needed - the filename is the source of truth

**Metric Calculation:**
- **True Positive (TP)**: Tool detected the expected CWE (e.g., `bandit_matches_expected = True`)
- **False Negative (FN)**: Tool did NOT detect the expected CWE (e.g., `bandit_matches_expected = False`)
- **False Positives**: We use the `*_matches_expected` boolean columns for metric calculations

**Handling Additional CWEs:**
- Tools may detect **additional CWEs beyond the expected one**
- These are tracked in `additional_cwes_detected` column for exploratory analysis
- **Important**: Additional CWEs are **NOT penalized** as false positives in metric computation
- Rationale: SecurityEval test cases may contain multiple vulnerabilities; we only validate detection of the primary expected CWE
- This follows the SecurityEval evaluation methodology where the focus is on detecting the targeted vulnerability

**Summary:**
- Use `*_matches_expected` columns for calculating Precision, Recall, F1-Score
- Analyze `additional_cwes_detected` separately to understand tool coverage
- This approach prevents unfair penalization for detecting legitimate vulnerabilities not in the expected set

### Validation and Summary

In [35]:
print("\n" + "=" * 80)
print("GROUND TRUTH LABELING SUMMARY")
print("=" * 80)

# Overall counts
total_workflows = len(df_labeled)
labeled_workflows = df_labeled['gt_source'].notna().sum()
unlabeled_workflows = total_workflows - labeled_workflows

print(f"\nTotal workflows: {total_workflows}")
print(f"Labeled workflows: {labeled_workflows} ({labeled_workflows/total_workflows*100:.1f}%)")
print(f"Unlabeled workflows: {unlabeled_workflows} ({unlabeled_workflows/total_workflows*100:.1f}%)")

# By prompt type
print("\nLabeling by prompt type:")
for prompt_type in ['Manual', 'SecurityEval']:
    mask = df_labeled['prompt_type'] == prompt_type
    total = mask.sum()
    labeled = (df_labeled[mask]['gt_source'].notna()).sum()
    print(f"  {prompt_type}: {labeled}/{total} labeled ({labeled/total*100:.1f}% if total > 0 else 0)")

# Vulnerability distribution
print("\nGround truth vulnerability distribution:")
vulnerable_count = df_labeled[df_labeled['gt_source'].notna()]['gt_vulnerable'].sum()
not_vulnerable_count = labeled_workflows - vulnerable_count
print(f"  Vulnerable: {vulnerable_count} ({vulnerable_count/labeled_workflows*100:.1f}% if labeled_workflows > 0 else 0)")
print(f"  Not vulnerable: {not_vulnerable_count} ({not_vulnerable_count/labeled_workflows*100:.1f}% if labeled_workflows > 0 else 0)")

# CWE statistics
print("\nCWE statistics:")
labeled_df = df_labeled[df_labeled['gt_source'].notna()]
with_cwes = sum(1 for cwes in labeled_df['gt_cwes'] if len(cwes) > 0)
total_cwes = sum(len(cwes) for cwes in labeled_df['gt_cwes'])
print(f"  Workflows with CWE labels: {with_cwes}")
print(f"  Total CWE labels: {total_cwes}")
if with_cwes > 0:
    print(f"  Average CWEs per vulnerable workflow: {total_cwes/with_cwes:.2f}")

# SecurityEval-specific metrics
print("\n" + "-" * 80)
print("SecurityEval Match Metrics:")
print("-" * 80)
securityeval_labeled = df_labeled[
    (df_labeled['prompt_type'] == 'SecurityEval') & 
    (df_labeled['expected_cwe'].notna()) & 
    (df_labeled['expected_cwe'] != '')
]

if len(securityeval_labeled) > 0:
    print(f"\nSecurityEval workflows with expected CWE: {len(securityeval_labeled)}")
    
    # Match rates
    print(f"\nTool Detection of Expected CWE:")
    print(f"  Bandit: {securityeval_labeled['bandit_matches_expected'].sum()} / {len(securityeval_labeled)} ({securityeval_labeled['bandit_matches_expected'].mean()*100:.1f}%)")
    print(f"  Semgrep: {securityeval_labeled['semgrep_matches_expected'].sum()} / {len(securityeval_labeled)} ({securityeval_labeled['semgrep_matches_expected'].mean()*100:.1f}%)")
    print(f"  AST: {securityeval_labeled['ast_matches_expected'].sum()} / {len(securityeval_labeled)} ({securityeval_labeled['ast_matches_expected'].mean()*100:.1f}%)")
    print(f"  Combined: {securityeval_labeled['combined_matches_expected'].sum()} / {len(securityeval_labeled)} ({securityeval_labeled['combined_matches_expected'].mean()*100:.1f}%)")
    
    # Additional CWEs
    total_additional = sum(len(cwes) for cwes in securityeval_labeled['additional_cwes_detected'])
    workflows_with_additional = sum(1 for cwes in securityeval_labeled['additional_cwes_detected'] if len(cwes) > 0)
    
    print(f"\nAdditional CWEs Detected (exploratory findings):")
    print(f"  Total additional CWEs: {total_additional}")
    print(f"  Workflows with additional CWEs: {workflows_with_additional}")
    print(f"  Note: These are NOT counted as false positives")
else:
    print("\n⚠️  No SecurityEval workflows with expected CWE labels")

print("\n" + "=" * 80)


GROUND TRUTH LABELING SUMMARY

Total workflows: 84
Labeled workflows: 84 (100.0%)
Unlabeled workflows: 0 (0.0%)

Labeling by prompt type:
  Manual: 42/42 labeled (100.0% if total > 0 else 0)
  SecurityEval: 42/42 labeled (100.0% if total > 0 else 0)

Ground truth vulnerability distribution:
  Vulnerable: 29 (34.5% if labeled_workflows > 0 else 0)
  Not vulnerable: 55 (65.5% if labeled_workflows > 0 else 0)

CWE statistics:
  Workflows with CWE labels: 63
  Total CWE labels: 63
  Average CWEs per vulnerable workflow: 1.00

--------------------------------------------------------------------------------
SecurityEval Match Metrics:
--------------------------------------------------------------------------------

SecurityEval workflows with expected CWE: 42

Tool Detection of Expected CWE:
  Bandit: 3 / 42 (7.1%)
  Semgrep: 3 / 42 (7.1%)
  AST: 3 / 42 (7.1%)
  Combined: 7 / 42 (16.7%)

Additional CWEs Detected (exploratory findings):
  Total additional CWEs: 68
  Workflows with additional

## 3. Export Labeled Dataset and Labeling Candidates

### Export Evaluation Labeled Dataset

In [36]:
# Convert list columns to string for CSV export
df_export = df_labeled.copy()
list_columns = ['gt_cwes', 'bandit_cwes', 'semgrep_cwes', 'ast_cwes', 'combined_cwes', 'all_cwes_across_workflow', 'additional_cwes_detected']
for col in list_columns:
    if col in df_export.columns:
        df_export[col] = df_export[col].apply(lambda x: json.dumps(x) if isinstance(x, list) else '[]')

# Export labeled dataset
labeled_path = DATA_DIR / 'evaluation_labeled.csv'
df_export.to_csv(labeled_path, index=False)

print(f"✓ Labeled dataset exported to: {labeled_path}")
print(f"  Total records: {len(df_export)}")
print(f"  Labeled records: {df_export['gt_source'].notna().sum()}")
print(f"\n  New columns added:")
print(f"    - expected_cwe (SecurityEval only)")
print(f"    - bandit_cwes, semgrep_cwes, ast_cwes, combined_cwes (initial detection)")
print(f"    - all_cwes_across_workflow (initial + all iterations)")
print(f"    - bandit_matches_expected, semgrep_matches_expected, ast_matches_expected, combined_matches_expected")
print(f"    - expected_cwe_found_anywhere (TRUE if found at any point in workflow)")
print(f"    - additional_cwes_detected")

✓ Labeled dataset exported to: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\evaluation_labeled.csv
  Total records: 84
  Labeled records: 84

  New columns added:
    - expected_cwe (SecurityEval only)
    - bandit_cwes, semgrep_cwes, ast_cwes, combined_cwes (initial detection)
    - all_cwes_across_workflow (initial + all iterations)
    - bandit_matches_expected, semgrep_matches_expected, ast_matches_expected, combined_matches_expected
    - expected_cwe_found_anywhere (TRUE if found at any point in workflow)
    - additional_cwes_detected


### Create Labeling Candidates for Manual Workflows

In [37]:
def extract_system_detected_cwes(row) -> List[str]:
    """
    Extract all system-detected CWEs from initial detection columns.
    
    Args:
        row: DataFrame row
        
    Returns:
        List of unique CWE IDs
    """
    all_cwes = set()
    
    cwe_columns = [
        'initial_detection_bandit_cwes',
        'initial_detection_semgrep_cwes',
        'initial_detection_ast_cwes'
    ]
    
    for col in cwe_columns:
        if col in row.index:
            cwes = parse_cwe_list(row[col])
            all_cwes.update(cwes)
    
    return sorted(list(all_cwes))


# Get manual workflows
df_manual = df_labeled[df_labeled['prompt_type'] == 'Manual'].copy()

print(f"Creating labeling candidates from {len(df_manual)} Manual workflows...")

# Extract system-detected CWEs
df_manual['system_detected_cwes'] = df_manual.apply(extract_system_detected_cwes, axis=1)

# Separate workflows with and without detected vulnerabilities
df_manual_with_vulns = df_manual[df_manual['vulnerabilities_found'] > 0]
df_manual_zero_vulns = df_manual[df_manual['vulnerabilities_found'] == 0]

print(f"  Workflows with detected vulnerabilities: {len(df_manual_with_vulns)}")
print(f"  Workflows with zero vulnerabilities: {len(df_manual_zero_vulns)}")

# Sample zero-vulnerability workflows
sample_size = min(MANUAL_ZERO_VULN_SAMPLE_SIZE, len(df_manual_zero_vulns))
df_manual_zero_sample = df_manual_zero_vulns.sample(
    n=sample_size,
    random_state=RANDOM_SEED
) if sample_size > 0 else pd.DataFrame()

print(f"  Sampled zero-vulnerability workflows: {len(df_manual_zero_sample)}")

# Combine for labeling candidates
df_candidates = pd.concat([df_manual_with_vulns, df_manual_zero_sample], ignore_index=True)

print(f"\nTotal labeling candidates: {len(df_candidates)}")

Creating labeling candidates from 42 Manual workflows...
  Workflows with detected vulnerabilities: 13
  Workflows with zero vulnerabilities: 29
  Sampled zero-vulnerability workflows: 10

Total labeling candidates: 23


### Export Labeling Candidates

In [38]:
# Select columns for labeling
candidate_columns = [
    'workflow_id',
    'prompt',
    'llm_response',
    'vulnerabilities_found',
    'system_detected_cwes',
    'total_vulnerabilities_identified',
    'total_vulnerabilities_fixed',
    'total_vulnerabilities_remaining'
]

# Filter to available columns
available_columns = [col for col in candidate_columns if col in df_candidates.columns]
df_candidates_export = df_candidates[available_columns].copy()

# Convert list to string for CSV
if 'system_detected_cwes' in df_candidates_export.columns:
    df_candidates_export['system_detected_cwes'] = df_candidates_export['system_detected_cwes'].apply(
        lambda x: json.dumps(x) if isinstance(x, list) else '[]'
    )

# Export candidates
candidates_path = OUTPUT_DIR / 'labeling_candidates_manual.csv'
df_candidates_export.to_csv(candidates_path, index=False)

print(f"✓ Labeling candidates exported to: {candidates_path}")
print(f"  Total candidates: {len(df_candidates_export)}")
print(f"  With detected vulnerabilities: {(df_candidates_export['vulnerabilities_found'] > 0).sum()}")
print(f"  Without detected vulnerabilities: {(df_candidates_export['vulnerabilities_found'] == 0).sum()}")

✓ Labeling candidates exported to: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\ground_truth\labeling_candidates_manual.csv
  Total candidates: 23
  With detected vulnerabilities: 13
  Without detected vulnerabilities: 10


## Summary Report

In [39]:
print("\n" + "=" * 80)
print("GROUND TRUTH ANNOTATION COMPLETE")
print("=" * 80)

print("\nGenerated outputs:")
print(f"  1. Labeled dataset: {DATA_DIR / 'evaluation_labeled.csv'}")
print(f"     - Total workflows: {len(df_labeled)}")
print(f"     - Labeled: {df_labeled['gt_source'].notna().sum()}")
print(f"     - Unlabeled: {(~df_labeled['gt_source'].notna()).sum()}")

print(f"\n  2. Labeling candidates: {OUTPUT_DIR / 'labeling_candidates_manual.csv'}")
print(f"     - Total candidates: {len(df_candidates_export)}")
print(f"     - For human review and annotation")

print("\n" + "-" * 80)
print("Key Features:")
print("-" * 80)
print("  ✓ SecurityEval ground truth extracted from filename (no external file needed)")
print("  ✓ Expected CWE detection tracked via *_matches_expected columns")
print("  ✓ Additional CWEs tracked separately (not counted as false positives)")
print("  ✓ Ready for metric calculation using match columns")

print("\nNext steps:")
if unlabeled_workflows > 0:
    manual_unlabeled = df_labeled[
        (df_labeled['prompt_type'] == 'Manual') & 
        (~df_labeled['gt_source'].notna())
    ].shape[0]
    
    if manual_unlabeled > 0:
        print(f"  1. Review and label {manual_unlabeled} unlabeled Manual workflows")
        print(f"  2. Update manual labels file: {MANUAL_LABELS}")
        print(f"  3. Re-run this notebook to update evaluation_labeled.csv")
    else:
        print("  ✓ All Manual workflows are labeled!")
        print("  ✓ All SecurityEval workflows auto-labeled from filename")
        print("  Ready for evaluation analysis")
else:
    print("  ✓ All workflows are labeled!")
    print("  Ready for evaluation analysis in next notebook (04_rq2_detection_metrics.ipynb)")

print("\n" + "=" * 80)


GROUND TRUTH ANNOTATION COMPLETE

Generated outputs:
  1. Labeled dataset: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\evaluation_labeled.csv
     - Total workflows: 84
     - Labeled: 84
     - Unlabeled: 0

  2. Labeling candidates: d:\Vincy-Certificates\AIDA\Winter'25\Thesis\Prototype\Notebooks\data\ground_truth\labeling_candidates_manual.csv
     - Total candidates: 23
     - For human review and annotation

--------------------------------------------------------------------------------
Key Features:
--------------------------------------------------------------------------------
  ✓ SecurityEval ground truth extracted from filename (no external file needed)
  ✓ Expected CWE detection tracked via *_matches_expected columns
  ✓ Additional CWEs tracked separately (not counted as false positives)
  ✓ Ready for metric calculation using match columns

Next steps:
  ✓ All workflows are labeled!
  Ready for evaluation analysis in next notebook (04_rq2_detection_

## Additional Analysis

In [40]:
# Show distribution of labeled vs unlabeled by prompt type
print("\nDetailed labeling status by prompt type:")
print("=" * 60)

for prompt_type in df_labeled['prompt_type'].unique():
    df_type = df_labeled[df_labeled['prompt_type'] == prompt_type]
    total = len(df_type)
    labeled = df_type['gt_source'].notna().sum()
    unlabeled = total - labeled
    
    print(f"\n{prompt_type}:")
    print(f"  Total: {total}")
    print(f"  Labeled: {labeled} ({labeled/total*100:.1f}%)")
    print(f"  Unlabeled: {unlabeled} ({unlabeled/total*100:.1f}%)")
    
    if labeled > 0:
        vulnerable = df_type[df_type['gt_source'].notna()]['gt_vulnerable'].sum()
        print(f"  Ground truth vulnerable: {vulnerable} ({vulnerable/labeled*100:.1f}%)")
        print(f"  Ground truth not vulnerable: {labeled - vulnerable} ({(labeled-vulnerable)/labeled*100:.1f}%)")


Detailed labeling status by prompt type:

SecurityEval:
  Total: 42
  Labeled: 42 (100.0%)
  Unlabeled: 0 (0.0%)
  Ground truth vulnerable: 8 (19.0%)
  Ground truth not vulnerable: 34 (81.0%)

Manual:
  Total: 42
  Labeled: 42 (100.0%)
  Unlabeled: 0 (0.0%)
  Ground truth vulnerable: 21 (50.0%)
  Ground truth not vulnerable: 21 (50.0%)
